# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели <i>кредитного скоринга</i> — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Обзор данных

In [ ]:
# Открываем таблицу и изучаем общую информацию о данных
import pandas as pd

data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<b>Обратить внимание:</b> Пропущены значения в столбцах days_employed и total_income

In [ ]:
# Изучаем более подробную информацию о данных
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


<b>Первичные наблюдения:</b>
1. <i>children</i> 

Разброс количества детей: от -1 до 20. Отрицательное значение может быть связано с некорректным вводом данных - попробуем взять по модулю. В случае многодетной семьи "20" может быть достоверным значением, но лучше посмотреть на количественные показатели.

2. <i>days_employed</i> 

Есть отрицательные значения (ошибка ввода данных?) и экстремально большие (401755 дней > 1100 лет стажа)

3. <i>dob_years</i> 

В базу попал минимальный возраст клиента = 0 лет (практически пропущенное значение)

In [ ]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<b>Обратить внимание:</b> Привести написание значений в столбце education к строчному

### Шаг 2.1 Заполнение пропусков

In [ ]:
# Проверим пропуски в столбцах days_employed и total_income
days_missing_share = data['days_employed'].isna().sum()/len(data['days_employed']) # доля пропущенных значений в days_employed
income_missing_share = data['total_income'].isna().sum()/len(data['total_income']) # доля пропущенных значений в total_income

print(f'Доля пропущенных значений в столбце days_employed составляет {days_missing_share:.1%}')
print(f'Доля пропущенных значений в столбце total_income составляет {income_missing_share:.1%}')

Доля пропущенных значений в столбце days_employed составляет 10.1%
Доля пропущенных значений в столбце total_income составляет 10.1%


In [ ]:
days_missing_ratio = data['days_employed'].isna().mean() # доля пропущенных значений в days_employed
income_missing_ratio = data['total_income'].isna().mean() # доля пропущенных значений в total_income

print(f'Доля пропущенных значений в столбце days_employed составляет {days_missing_ratio:.1%}')
print(f'Доля пропущенных значений в столбце total_income составляет {income_missing_ratio:.1%}')

Доля пропущенных значений в столбце days_employed составляет 10.1%
Доля пропущенных значений в столбце total_income составляет 10.1%


Доли пропущенных значений в обоих столбцах совпадают.
<b>Гипотеза:</b> не пропущены ли данные в одних и тех же строках?

In [ ]:
data[(data['days_employed'].isna() == True) & (data['total_income'].isna() == True)].count()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

<b>Вывод:</b> похоже, действительно данные пропущены в одних и тех же строках. 

<b>Возможная причина появления пропусков в данных:</b> при невнесенных днях стажа заработок также не рассчитывается.

In [ ]:
# Заполним пропуски в столбце days_employed средними значениями по типу занятости:
data['days_employed'] = data.groupby('income_type')['days_employed'].fillna(data['days_employed'].mean())
# Заполним пропуски в столбце total_income медианным значением по типу занятости:
#data['total_income'] = data.groupby('income_type')['total_income'].fillna(data['total_income'].median())
#data.isna().sum()

In [ ]:
# Проверим медианные значения заработка по типу занятости
data['income_type'].value_counts() # анализ типов занятости

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [ ]:
# медианный заработок сотрудника
employee_income_median = data[data['income_type'] == 'сотрудник']['total_income'].median()
# медианный заработок компаньона
partner_income_median = data[data['income_type'] == 'компаньон']['total_income'].median()
# медианный заработок пенсионера
retired_income_median = data[data['income_type'] == 'пенсионер']['total_income'].median() 
# медианный заработок госслужащего
official_income_median = data[data['income_type'] == 'госслужащий']['total_income'].median()
# медианный заработок предпринимателя
business_income_median = data[data['income_type'] == 'предприниматель']['total_income'].median()
# медианный заработок безработного
idle_income_median = data[data['income_type'] == 'безработный']['total_income'].median()
# медианный заработок в декрете
maternity_income_median = data[data['income_type'] == 'в декрете']['total_income'].median()
# медианный заработок студента
student_income_median = data[data['income_type'] == 'студент']['total_income'].median()
print('Медианные заработки по типу занятости:')
print('Сотрудник:', employee_income_median)
print('Компаньон:', partner_income_median)
print('Пенсионер:', retired_income_median)
print('Госслужащий:', official_income_median)
print('Предприниматель:', business_income_median)
print('Безработный:', idle_income_median)
print('В декрете:', maternity_income_median)
print('Студент:', student_income_median)

Медианные заработки по типу занятости:
Сотрудник: 142594.39684740017
Компаньон: 172357.95096577113
Пенсионер: 118514.48641164352
Госслужащий: 150447.9352830068
Предприниматель: 499163.1449470857
Безработный: 131339.7516762103
В декрете: 53829.13072905995
Студент: 98201.62531401133


<div class="alert alert-block alert-info">
<b>Комментарий студента:</b> 
Медианные значение по каждому типу занятости выглядят достоверно. Заполним этими значениями пропуски по типу занятости.
</div>

In [ ]:
# Упорядочим процесс с помощью функции заполнения пропусков в столбце total_income медианным значением по типу занятости:
def income_fillna(income_type, category):
    condition = data['income_type'] == income_type
    data.loc[condition, 'total_income'] = data.loc[condition, 'total_income'].fillna(category['total_income'].median())

# Подготовим категории клиентов по типу заработка, по которым пройдемся функцией:
employee = data[data['income_type'] == 'сотрудник']                                          
partner = data[data['income_type'] == 'компаньон']  
retired = data[data['income_type'] == 'пенсионер']
official = data[data['income_type'] == 'госслужащий']
business = data[data['income_type'] == 'предприниматель']
idle = data[data['income_type'] == 'безработный']
maternity = data[data['income_type'] == 'в декрете']  
student = data[data['income_type'] == 'студент']
  
# Прогоним созданные переменные категорий заработка через функцию заполнения пропусков медианным значением income_fillna:                                           

income_fillna('сотрудник', employee)
income_fillna('компаньон', partner)
income_fillna('пенсионер', retired)
income_fillna('госслужащий', official)
income_fillna('госслужащий', business)
income_fillna('госслужащий', idle)
income_fillna('госслужащий', maternity)
income_fillna('госслужащий', student)                                               

data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        1
purpose             0
dtype: int64

<div class="alert alert-block alert-info">
<b>Комментарий студента:</b> 
Остался еще один пропуск. Медиана не везде сработала, видимо. Поищем, где произошло упущение
</div>

In [ ]:
# Перебором категорий обнаружен пропуск заработка среди предпринимателей
business.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,2.0,2.000000,2.00000,2.0,2.000000,2.0,1.000000
mean,0.0,31262.824789,42.50000,0.0,0.500000,0.0,499163.144947
std,0.0,44948.901238,21.92031,0.0,0.707107,0.0,NaN
min,0.0,-520.848083,27.00000,0.0,0.000000,0.0,499163.144947
25%,0.0,15370.988353,34.75000,0.0,0.250000,0.0,499163.144947
50%,0.0,31262.824789,42.50000,0.0,0.500000,0.0,499163.144947
75%,0.0,47154.661225,50.25000,0.0,0.750000,0.0,499163.144947
max,0.0,63046.497661,58.00000,0.0,1.000000,0.0,499163.144947


<div class="alert alert-block alert-info">
<b>Комментарий студента:</b> 
Судя по данным, в категории предпринимателей вообще только 2 строчки, а данные по заработку указаны только для одного клиента. Видимо, поэтому заполнение пропуска медианным значением и не сработало.
Поскольку не могу судить о заработке второго предпринимателя и выборки по категории нет, предпочитаю избавиться от этого пропуска удалением строки.
</div>

In [ ]:
data.drop(data.loc[(data['income_type'] == 'предприниматель') & (data['total_income'].isna())].index, inplace=True)
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

<div class="alert alert-block alert-info">
<b>Комментарий студента:</b> 
Наконец, везде по ноликам. Надеюсь, все в порядке.
</div>

In [ ]:
# Проанализируем замеченные ранее аномалии количественно.
print('Количество клиентов с указанием отрицательного стажа:', data[data['days_employed'] < 0]['days_employed'].count())
print('Количество клиентов с отрицательным числом детей:', data[data['children'] < 0]['children'].count())
print('Количество клиентов с нулевым возрастом:', data[data['dob_years'] == 0]['dob_years'].count())
print('Количество клиентов с 20 детьми:', data[data['children'] >= 20]['children'].count())
print('Количество клиентов с экстремальным стажем:', data[data['days_employed'] > 14600]['days_employed'].count()) 
# средний стаж до выхода на пенсию - 34.5 лет.

Количество клиентов с указанием отрицательного стажа: 15906
Количество клиентов с отрицательным числом детей: 47
Количество клиентов с нулевым возрастом: 101
Количество клиентов с 20 детьми: 76
Количество клиентов с экстремальным стажем: 5618


<b>Выводы</b>: 
1. Б<b>о</b>льшая часть значений 'days_employed' отрицательные. Такую долю значений удалять бессмысленно - попробуем взять значения по модулю.
2. Аналогично с отрицательным количеством детей. Возможно, при вводе значений было введено тире для форматирования. Возьмем значения по модулю.
3. Нулевой возраст - аномалия сродни пропущенному значению. Эти данные могли быть попросту не внесены.
4. Экстремально многодетные семьи - большая роскошь. Может быть, при вводе данных случайно добавился "0". Попробуем исправить или учесть при категоризации.
5. Зашкаливающий стаж у такой большой доли клиентов также кажется ошибкой.

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [ ]:
# Приведем столбец education к строчному формату
data['education'] = data['education'].str.lower()
# Также приведем к строчному формату столбец family_status
data['family_status'] = data['family_status'].str.lower()

In [ ]:
# Данные с отрицательным количеством детей предпочитаю взять по модулю, чтобы не терять драгоценные значения "-1"
data['children'] = data['children'].abs()
# Отрицательные значения стажа также беру по модулю
data['days_employed'] = data['days_employed'].abs()
# Проверка количества строк с отрицательным количеством детей
print ('Клиентов с отрицательным количеством детей', data[data['children'] < 0]['children'].count())

Клиентов с отрицательным количеством детей 0


In [ ]:
# Данные с указанием нулевого возраста предлагаю заменить на средние значения по типу занятости (ориентир - студент)
data['dob_years'] = data['dob_years'].replace(0, int(data['dob_years'].mean()))
# Проверка количества строк с нулевым значением возраста
print('Клиентов с нулевым возрастом', data[data['dob_years'] == 0]['dob_years'].count())

Клиентов с нулевым возрастом 0


In [ ]:
# Проверим общие данные датафрейма после манипуляций
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21524.000000,21524.000000,21524.000000,21524.000000,21524.000000,21524.000000,2.152400e+04
mean,0.543301,66524.203610,43.494471,0.817274,0.972589,0.080886,1.652098e+05
std,1.379903,131830.933030,12.218097,0.548123,1.420341,0.272667,9.801952e+04
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1025.593536,34.000000,1.000000,0.000000,0.000000,1.077960e+05
50%,0.000000,2609.705632,43.000000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,63046.497661,53.000000,1.000000,1.000000,0.000000,1.955452e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


<b>Вывод</b>

Явные аномалии удалены. Данные выглядят более адекватными.

<div class="alert alert-block alert-info">
<b>Комментарий студента:</b> 
Если предположить гипотезу, что в аномальную численность детей закралась опечатка (скажем, дополнительный "0"), произведем замену "20" на "2".
</div>

In [ ]:
data['children'] = data['children'].replace(20, 2)
data.groupby('children')['children'].count()

children
0    14148
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

<div class="alert alert-block alert-info">
<b>Комментарий студента:</b> 
Аномальная численность детей исключена из выборки.
</div>

<div class="alert alert-block alert-info">
<b>Комментарий студента:</b> 
Изучим столбец с гендерной принадлежностью клиентов.
</div>

In [ ]:
data.groupby('gender')['gender'].count()

gender
F      14236
M       7287
XNA        1
Name: gender, dtype: int64

<div class="alert alert-block alert-info">
<b>Комментарий студента:</b> 
Женщин-клиентов почти вдвое больше, чем мужчин-клиентов. Пропусков не выявлено, но есть подозрительное значение XNA. Поскольку оно встречается всего один раз, попробуем этой строкой пренебречь и исключить из выборки.
</div>

In [ ]:
data.drop(data.loc[data['gender'] == 'XNA'].index, inplace=True)
data.groupby('gender')['gender'].count()

gender
F    14236
M     7287
Name: gender, dtype: int64

<div class="alert alert-block alert-info">
<b>Комментарий студента:</b> 
Явные аномалии исключены. Попробуем поработать с данными дальше.
</div>

### Шаг 2.3. Изменение типов данных.

In [ ]:
# Заменим вещественный тип данных в столбце total_income на целочисленный с помощью метода astype()
data['total_income'] = data['total_income'].astype('int')
# Проверим типы данных в датафрейма
data.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income          int64
purpose              object
dtype: object

<b>Вывод</b>
В целом типы данных для наших целей везде выглядят корректно.

### Шаг 2.4. Удаление дубликатов.

В данных присутствуют строки-дубликаты, удалим их. Также обработаем неявные дубликаты. Например, в столбце education есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к одному регистру. Проверим остальные столбцы. После удаления дубликатов сделаем следующее:
- поясним, как выбрали метод для поиска и удаления дубликатов в данных
- приведем возможные причины появления дубликатов

In [ ]:
# Проверим суммарное число дубликатов по датафрейму:
data.duplicated().sum()

71

In [ ]:
# Изучим данные по дубликатам
data[data.duplicated(keep=False)].sort_values(by=['total_income', 'days_employed'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1005,0,63046.497661,62,среднее,1,женат / замужем,0,F,пенсионер,0,118514,ремонт жилью
1191,0,63046.497661,61,среднее,1,женат / замужем,0,F,пенсионер,0,118514,операции с недвижимостью
1511,0,63046.497661,58,высшее,0,не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование
1681,0,63046.497661,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
2052,0,63046.497661,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
16148,0,63046.497661,45,среднее,1,гражданский брак,1,F,компаньон,0,172357,свадьба
17379,0,63046.497661,54,высшее,0,женат / замужем,0,M,компаньон,0,172357,операции с коммерческой недвижимостью
17774,1,63046.497661,40,среднее,1,гражданский брак,1,F,компаньон,0,172357,строительство жилой недвижимости
19369,0,63046.497661,45,среднее,1,гражданский брак,1,F,компаньон,0,172357,свадьба


<b>Вывод:</b> действительно похоже на явные дубликаты, вероятно, по причине задвоения данных. Думаю, их можно смело удалить.

In [ ]:
# Удаляем дубликаты
data = data.drop_duplicates()

In [ ]:
# Проверим удаление дубликатов
data.duplicated().sum()

0

<b>Вывод:</b> явные дубликаты удалены.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма со столбцами:
- education_id и education - в первом;
- family_status_id и family_status - во втором.

In [ ]:
data_dict_one = data[['education', 'education_id']]
print(data_dict_one.head(10))
data_dict_two = data[['family_status', 'family_status_id']]
print(data_dict_two.head(10))

  education  education_id
0    высшее             0
1   среднее             1
2   среднее             1
3   среднее             1
4   среднее             1
5    высшее             0
6    высшее             0
7   среднее             1
8    высшее             0
9   среднее             1
      family_status  family_status_id
0   женат / замужем                 0
1   женат / замужем                 0
2   женат / замужем                 0
3   женат / замужем                 0
4  гражданский брак                 1
5  гражданский брак                 1
6   женат / замужем                 0
7   женат / замужем                 0
8  гражданский брак                 1
9   женат / замужем                 0


In [ ]:
# Удалим из исходного датафрейма столбцы education и family_status, 
# оставив только их идентификаторы: education_id и family_status_id. 
# Новые датафреймы - это те самые "словари" (не путаем с одноименной структурой данных в Python), 
# к которым  мы сможем обращаться по идентификатору.

data_new = data.drop(['education', 'family_status'], axis=1)
print(data_new.describe())

           children  days_employed     dob_years  education_id  \
count  21452.000000   21452.000000  21452.000000  21452.000000   
mean       0.480608   66538.704949     43.473895      0.817080   
std        0.756090  132051.112971     12.212950      0.548612   
min        0.000000      24.141633     19.000000      0.000000   
25%        0.000000    1023.688788     33.000000      1.000000   
50%        0.000000    2591.855682     43.000000      1.000000   
75%        1.000000   63046.497661     53.000000      1.000000   
max        5.000000  401755.400475     75.000000      4.000000   

       family_status_id          debt  total_income  
count      21452.000000  21452.000000  2.145200e+04  
mean           0.973942      0.081158  1.653022e+05  
std            1.421618      0.273084  9.816507e+04  
min            0.000000      0.000000  2.066700e+04  
25%            0.000000      0.000000  1.076082e+05  
50%            0.000000      0.000000  1.425940e+05  
75%            1.000000    

<div class="alert alert-block alert-info">
<b>Комментарий студента:</b> 
Удалим дубликаты из таблиц-словарей.
</div>

In [ ]:
data_dict_one = data_dict_one.drop_duplicates().reset_index(drop=True)
print(data_dict_one.head()) 

             education  education_id
0               высшее             0
1              среднее             1
2  неоконченное высшее             2
3            начальное             3
4       ученая степень             4


In [ ]:
data_dict_two = data_dict_two.drop_duplicates().reset_index(drop=True)
print(data_dict_two.head()) 

           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  не женат / не замужем                 4


### Шаг 2.6. Категоризация дохода.

На основании диапазонов, указанных ниже, создадим столбец total_income_category с категориями:

- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

In [ ]:
def total_income_category(row):
    if row['total_income'] <= 30000:
        return 'E'
    elif 30000 < row['total_income'] <= 50000:
        return 'D'
    elif 50000 < row['total_income'] <= 200000:
        return 'C'
    elif 200000 < row['total_income'] <= 1000000:
        return 'B'
    else:
        return 'A'

In [ ]:
data_new['total_income_category'] = data_new.apply(total_income_category, axis=1)
data_new['total_income_category'].value_counts()

C    16015
B     5040
D      350
A       25
E       22
Name: total_income_category, dtype: int64

<b>Вывод:</b> самая распространенная категория клиентов - с доходом от 50000 рублей до 200000 рублей.

### Шаг 2.7. Категоризация целей кредита.

Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:

- 'операции с автомобилем',

- 'операции с недвижимостью',

- 'проведение свадьбы',

- 'получение образования'.

In [ ]:
# Сначала проанализируем список уникальных целей из столбца purpose
data_new.groupby('purpose')['purpose'].count()

purpose
автомобили                                478
автомобиль                                494
высшее образование                        452
дополнительное образование                460
жилье                                     646
заняться высшим образованием              496
заняться образованием                     408
на покупку автомобиля                     471
на покупку подержанного автомобиля        478
на покупку своего автомобиля              505
на проведение свадьбы                     768
недвижимость                              633
образование                               447
операции с жильем                         652
операции с коммерческой недвижимостью     650
операции с недвижимостью                  675
операции со своей недвижимостью           627
покупка жилой недвижимости                605
покупка жилья                             646
покупка жилья для сдачи                   651
покупка жилья для семьи                   638
покупка коммерческой недви

Действительно заметны различные написания одной и той же цели. Напишем функцию для категоризации целей кредита.
Создадим словарь с исходными ключами:

In [ ]:
def purpose_category(row):
    try:
        if 'автомоб' in row['purpose']:
            return 'операции с автомобилем'
        elif 'недвижимост' or 'жил' in row['purpose']:
            return 'операции с недвижимостью'
        elif 'свад' in row['purpose']:
            return 'проведение свадьбы'
        elif 'образован' in row['purpose']:
            return 'получение образования'
    except:
        return 'Ошибка!'

In [ ]:
data_new['purpose_category'] = data_new.apply(purpose_category, axis=1)
data_new['purpose_category'].value_counts()

операции с недвижимостью    17146
операции с автомобилем       4306
Name: purpose_category, dtype: int64

<b>Промежуточный итог:</b> странно, но простая функция не сработала и не распознала все цели, выдав только две из них. Придется углубиться в проблему и все-таки изучить метод лемматизации, указанный в Notion.

In [ ]:
from pymystem3 import Mystem

m = Mystem()
purpose_lemma = []
for i in data_new['purpose']:
    result = ''.join(m.lemmatize(i)).strip()
    purpose_lemma.append(result)
data_new['purpose_lemma'] = purpose_lemma

def purpose_category(data):
    if 'автомобиль' in data:
        return 'операции с автомобилем'
    elif ('недвижимость' in data) or ('жилье' in data):
        return 'операции с недвижимостью'
    elif 'свадьба' in data:
        return 'проведение свадьбы'
    elif 'образование' in data:
        return 'получение образования'
    else:
        return 'прочее'

data_new['purpose_category'] = data_new['purpose_lemma'].apply(purpose_category)
data_new.drop(['purpose_lemma'], axis='columns', inplace=True)

data_new.groupby('purpose_category')['purpose_category'].count()

purpose_category
операции с автомобилем       4306
операции с недвижимостью    10809
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

<b>Вывод:</b> Получена выгрузка по всем целям. Большинство клиентов берут кредит для операций с недвижимостью.

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [ ]:
# Оформим сводную таблицу по количеству детей и задолженностям по выплате кредита
debt_child_pivot = data_new.pivot_table(index='children', columns='debt', values='education_id', aggfunc='count')
debt_child_pivot

debt,0,1
children,,
0,13026.0,1063.0
1,4410.0,445.0
2,1926.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


In [ ]:
# Посчитаем доли невыплативших кредит по количеству детей
#zero_kids = 1063/(13028+1063)
#one_kid = 445/(4410+445)
#two_kids = 194/(1858+194)
#three_kids = 27/(303+27)
#four_kids = 4/(37+4)
#many_kids = 8/(68+8)
#print(f'Доля невыплативших кредит среди клиентов без детей: {zero_kids:.1%}')
#print(f'Доля невыплативших кредит среди клиентов с 1 ребенком: {one_kid:.1%}')
#print(f'Доля невыплативших кредит среди клиентов с 2 детьми: {two_kids:.1%}')
#print(f'Доля невыплативших кредит среди клиентов с 3 детьми: {three_kids:.1%}')
#print(f'Доля невыплативших кредит среди клиентов с 4 детьми: {four_kids:.1%}')
#print(f'Доля невыплативших кредит среди многодетных клиентов: {many_kids:.1%}')

<div class="alert alert-block alert-info">
<b>Комментарий студента:</b> 
Да, ручной ввод - это скверно. Общий психологический фон отключил рацио и заставил вбивать все руками. Попробую переделать, спасибо.
</div>

In [ ]:
debt_child_pivot['stats'] = round((debt_child_pivot[1] / (debt_child_pivot[1] + debt_child_pivot[0]) * 100), 2)
debt_child_pivot

debt,0,1,stats
children,,,
0,13026.0,1063.0,7.54
1,4410.0,445.0,9.17
2,1926.0,202.0,9.49
3,303.0,27.0,8.18
4,37.0,4.0,9.76
5,9.0,NaN,NaN


##### Вывод 1:

Клиенты без детей быстрее выплачивают кредит и не склонны к просрочке. Чем больше детей, тем сложнее выплачивать кредит.

In [ ]:

data_new.pivot_table(index='children', values='debt', aggfunc=['count', 'sum', 'mean'])

,count,sum,mean
,debt,debt,debt
children,,,
0,14089,1063,0.075449
1,4855,445,0.091658
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
# Оформим сводную таблицу по семейному положению и задолженностям по выплате кредита
debt_family_pivot = data_new.pivot_table(index='family_status_id', columns='debt', values='income_type', aggfunc='count')
debt_family_pivot

debt,0,1
family_status_id,,
0,11407,931
1,3762,388
2,896,63
3,1110,85
4,2536,274


In [ ]:
# Посчитаем доли невыплативших кредит по семейному статусу
debt_family_pivot['stats'] = round((debt_family_pivot[1] / (debt_family_pivot[1] + debt_family_pivot[0]) * 100), 2)
debt_family_pivot

debt,0,1,stats
family_status_id,,,
0,11407,931,7.55
1,3762,388,9.35
2,896,63,6.57
3,1110,85,7.11
4,2536,274,9.75


##### Вывод 2: 

Клиенты, не связанные узами официального брака или находящиеся в гражданском браке, чаще не выплачивают кредиты в срок.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
# Оформим сводную таблицу по категориям дохода и задолженностям по выплате кредита
debt_income_pivot = data_new.pivot_table(index='total_income_category', columns='debt', values='income_type', aggfunc='count')
debt_income_pivot

debt,0,1
total_income_category,,
A,23,2
B,4684,356
C,14655,1360
D,329,21
E,20,2


In [ ]:
# Посчитаем доли невыплативших кредит по категориям дохода
debt_income_pivot['stats'] = round((debt_income_pivot[1] / (debt_income_pivot[1] + debt_income_pivot[0]) * 100), 2)
debt_income_pivot

debt,0,1,stats
total_income_category,,,
A,23,2,8.00
B,4684,356,7.06
C,14655,1360,8.49
D,329,21,6.00
E,20,2,9.09


##### Вывод 3: 

Самыми ответственными и обязательными при выплате кредита оказывается категория клиентов с доходом от 30000 до 50000 рублей.

##### Вопрос 4: Как разные цели влияют на его возврат в срок?

In [ ]:
# Оформим сводную таблицу по категориям цели кредита и задолженностям по выплате кредита
debt_purpose_pivot = data_new.pivot_table(index='purpose_category', columns='debt', values='income_type', aggfunc='count')
debt_purpose_pivot

debt,0,1
purpose_category,,
операции с автомобилем,3903,403
операции с недвижимостью,10027,782
получение образования,3643,370
проведение свадьбы,2138,186


In [ ]:
# Посчитаем доли невыплативших кредит по категориям цели кредита
debt_purpose_pivot['stats'] = round((debt_purpose_pivot[1] / (debt_purpose_pivot[1] + debt_purpose_pivot[0]) * 100), 2)
debt_purpose_pivot

debt,0,1,stats
purpose_category,,,
операции с автомобилем,3903,403,9.36
операции с недвижимостью,10027,782,7.23
получение образования,3643,370,9.22
проведение свадьбы,2138,186,8.00


##### Вывод 4: 

Клиенты, берущие кредит для приобретения/проведение операций с жильем, более обязательны в выплате кредита в срок.

### Общий вывод:

1. Семейное положение и количество детей влияет на факт погашения кредита в срок: клиенты с официально оформленным браком (или бывших в нем) или не имеющие детей - самые ответственные при выплате кредита.

2. Нет прямой зависимости между уровнем дохода и возвратом кредита в срок: самая обязательная категория клиентов в погашении - с уровнем дохода от 30 до 50 тыс. рублей.

3. Возврат кредита зависит от цели получения кредита: клиенты, берущие в кредит средства для операций с недвижимостью, более ответственны при погашении.

## Итоговый вывод: 
Проанализированы данные о платёжеспособности 21 525 клиентов с учетом следующих факторов: количество детей в семье, трудовой стаж, возраст клиента, уровень образования, семейное положение, пол клиента, тип занятости, кредитный опыт по возврату/невозврату займов, общий месячный доход, цель получения кредита.

Проведена обработка данных оригинального датасета с исключением пропусков, дубликатов и аномалий.
    
Проанализированы категории данных по целям получения кредита: недвижимость, образование, автомобиль и свадьба.
Также была проанализирована зависимость возврата кредита от уровня дохода с разбиением на 5 категорий уровня заработка.
    
Итак, изучена зависимость между возвратом кредита в срок и:
    
    - наличием детей
    
    - семейным положением
    
    - уровнем дохода
    
    - целями получения кредита
    

Наиболее явная связь возврата кредита прослеживается в зависимости от семейного положения и цели получения кредита. Самые ответственнные заемщики:
    
    - находящиеся или находившиеся в официальном браке
    
    - берущие кредит для операций с недвижимостью.
</div>